<a href="https://colab.research.google.com/github/VictorLA12/Recomenda-Filmes/blob/main/Introdu%C3%A7%C3%A3o_%C3%A0_recomenda%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
filmes = pd.read_csv("movies.csv")
filmes.columns = ['FilmeId', 'Título', 'Gênero']
filmes = filmes.set_index('FilmeId')
filmes.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
notas = pd.read_csv("ratings.csv")
notas.head()

In [ ]:
notas.columns = ['UsuárioId', 'FilmeId', 'Nota', 'Momento']
notas.head()

In [ ]:
notas.describe()

## Primeira tentativa de Recomendação


In [ ]:
total_de_votos = notas['FilmeId'].value_counts()
total_de_votos.head()


In [ ]:
filmes['Avaliações'] = total_de_votos
filmes.head()

In [ ]:
#ordenar por número de avaliações
#ascending = False é pra colocar em ordem descrescente
filmes.sort_values('Avaliações', ascending = False).head(10)

In [ ]:
#agrupar por nota
nota_média = notas.groupby('FilmeId').mean()["Nota"]
nota_média.head(20)

In [ ]:
filmes['Média'] = nota_média

In [ ]:
#filtrar filmes com mais de 50 avaliações e ordenar por média
filmes.query("Avaliações >= 50").sort_values('Média', ascending = False).head(10)
filmes_selecionados = filmes.query("Avaliações >= 50")

In [ ]:
filmes.query("Avaliações >= 50").sort_values('Média', ascending = False).head(10)


In [ ]:
filmes_selecionados.head(10)

In [ ]:
#Ubiraci (4, 5)
#Ubirajara (3.5, 2)
import matplotlib.pyplot as plt
plt.plot(4, 5, "go")
plt.plot(3.5, 2, "ro")
plt.legend(["Ubiraci", "Ubijara"])


In [ ]:
plt.title('Distância entre 2 usuários')
import numpy as np
Ubiraci = np.array([4, 5])
Ubirajara = np.array([3.5, 2])



In [ ]:
Ubiraci - Ubirajara

In [ ]:
from math import sqrt
def pitágoras (a,b):
  (deltax, deltay) = a - b
  return sqrt(deltax*deltax + deltay*deltay)

In [ ]:
pitágoras(Ubiraci,Ubirajara)

In [ ]:
def distancia (a,b):
  return np.linalg.norm(a-b)

In [ ]:
def notas_usuario(usuario):
  notas_usuario = notas.query("UsuárioId==%d" % usuario)
  notas_usuario[["FilmeId", "Nota"]].set_index('FilmeId')
  return notas_usuario


In [ ]:
notas_usuario(610)

In [ ]:
#join junta 2 dataframes pelo índice. Precisamos usar um sufixo para q as colunas não se sobeponham.
usuario1 = notas_usuario(1)
usuario10 = notas_usuario(10)
diferenca = usuario1.join(usuario10, lsuffix="_esquerda", rsuffix="_direita")
diferenca ['Nota_direita']

In [ ]:
distancia (diferenca ['Nota_esquerda'],diferenca ['Nota_direita'])


In [ ]:
def distancia_entre_usuarios (usuario_id1, usuario_id2):
  nota_1 = notas_usuario(usuario_id1)
  nota_2 = notas_usuario(usuario_id2)
  diferenca = nota_1.join(nota_2, lsuffix="_esquerda", rsuffix="_direita").dropna()
  dist =  distancia (diferenca ['Nota_esquerda'],diferenca ['Nota_direita'])
  return [usuario_id1, usuario_id2, dist]


In [ ]:
quantidade_de_usuarios = len(notas['UsuárioId'].unique())
print('Temos {} usuários'.format(quantidade_de_usuarios))

In [ ]:
def distancia_de_todos(voce_id):
  distancias = []
  for usuario_id in notas['UsuárioId'].unique():
    informacoes = distancia_entre_usuarios(voce_id, usuario_id)
    distancias.append(informacoes)
    distancias = pd.DataFrame(distancias, columns = ['voce', 'outro_usuario', 'distância'])
  return distancias


In [ ]:
distancia_de_todos(1)

,voce,outro_usuario,distância
0,1,1,0.0


##Usuários sem nada em comum são colocados distantes

In [ ]:
def distancia_entre_usuarios (usuario_id1, usuario_id2, mínimo = 5):
  nota_1 = notas_usuario(usuario_id1)
  nota_2 = notas_usuario(usuario_id2)
  diferenca = nota_1.join(nota_2, lsuffix="_esquerda", rsuffix="_direita").dropna()

  if (len(diferenca) < mínimo):
    return [usuario_id1, usuario_id2, 1000000]

  dist =  distancia (diferenca ['Nota_esquerda'],diferenca ['Nota_direita'])
  return [usuario_id1, usuario_id2, dist]


In [ ]:
def usuarios_mais_proximos (voce_id):
  dist_todos = distancia_de_todos(voce_id)
  dist_todos = dist_todos.sort_values('distância')
  dist_todos = dist_todos.set_index('outro_usuario').drop(voce_id)
  return dist_todos


In [ ]:
usuarios_mais_proximos (1).head()

In [ ]:
#n = número de usuários a analisar
def distancia_de_todos(voce_id, n = None):
  todos_usuarios = notas['UsuárioId'].unique()
  if n:
    todos_usuarios = todos_usuarios[:n]
  dist_todos = [distancia_entre_usuarios(voce_id, UsuárioId) for UsuárioId in todos_usuarios]
  dist_todos = list(filter(None, dist_todos))
  dist_todos = pd.DataFrame(dist_todos, columns = ['voce', 'outro_usuario', 'distância'])
  return dist_todos

In [ ]:
#n = número de usuários a analisar
def usuarios_mais_proximos (voce_id, n = None):
  dist_todos = distancia_de_todos(voce_id)
  dist_todos = dist_todos.sort_values('distância')
  dist_todos = dist_todos.set_index('outro_usuario').drop(voce_id)
  return dist_todos


In [ ]:
usuarios_mais_proximos(1, n = 50)

,voce,distância
outro_usuario,,


In [ ]:
#n = número de usuários a analisar
def distancia_entre_usuarios (usuario_id1, usuario_id2, mínimo = 5):
  nota_1 = notas_usuario(usuario_id1)
  nota_2 = notas_usuario(usuario_id2)
  diferenca = nota_1.join(nota_2, lsuffix="_esquerda", rsuffix="_direita").dropna()

  if (len(diferenca) < mínimo):
    return None

  dist =  distancia (diferenca ['Nota_esquerda'],diferenca ['Nota_direita'])
  return [usuario_id1, usuario_id2, dist]

In [ ]:
#sugestões de filmes pra vc
voce=1
def sugestoes(voce, n=None):
  notas_voce = notas_usuario(voce)
  filmes_vistos = notas_voce.index
  similares = usuarios_mais_proximos (voce, n = n)
  similar = similares.iloc[0].name
  notas_similar = notas_usuario(similar)
  notas_similar = notas_similar.drop(filmes_vistos, errors='ignore')
  recomendacoes = notas_similar.sort_values('nota', ascending = False)
  return recomendacoes.join(filmes)

##Sugerindo baseado em vários usuários

In [ ]:
def usuarios_mais_proximos (voce_id, mais_proximos = 10, n = None):
  dist_todos = distancia_de_todos(voce_id)
  dist_todos = dist_todos.sort_values('distância')
  dist_todos = dist_todos.set_index('outro_usuario').drop(voce_id)
  return dist_todos.head(mais_proximos)


In [ ]:
usuarios_mais_proximos (1, mais_proximos=2, n=100)

In [ ]:

#KNN = k-nearest neighbor
def sugestao(voce, mais_proximos=10, n=None):
  notas_voce = notas_usuario(voce)
  filmes_vistos = notas_voce.index
  similares = usuarios_mais_proximos (voce, mais_proximos= mais_proximos, n = n)
  usuarios_similares = similares.index
  notas_similares = notas.set_index('UsuárioId').loc[usuarios_similares]
  recomendacoes = notas_similares.groupby('FilmeId').mean()[["Nota"]]
  recomendacoes.sort_values("Nota", ascending = False)
  return recomendacoes.join(filmes)

In [ ]:
sugestao(1, n=50)

##Testar um usuário novo

In [ ]:
def novo_usuario(dados):
  novo_usuario = notas['UsuárioId'].max()+1
  notas_novo_usuario = pd.DataFrame(dados, columns = ['FilmeId', 'Nota'])
  notas_novo_usuario['UsuárioId'] = novo_usuario
  return pd.concat([notas, notas_novo_usuario])

In [ ]:
notas = novo_usuario([[12, 5], [1, 4.5], [1989, 4.9], [33, 4.8], [2012, 4.7], [2015, 4.6], [30, 4.4], [6, 4.3], [2013, 4.2], [7, 4.1], [9, 4]])
notas.tail()
#tail = 5 ultimos

,UsuárioId,FilmeId,Nota,Momento
6,611,30,4.4,NaN
7,611,6,4.3,NaN
8,611,2013,4.2,NaN
9,611,7,4.1,NaN
10,611,9,4.0,NaN
